In [17]:
import requests
import json
import re
from datetime import datetime

In [ ]:
today = datetime.now().strftime("%Y%m%d")
station_url = "https://indianrailways.gov.in/index/index_data/data.js"
trains_url = f"https://enquiry.indianrail.gov.in/mntes/javascripts/train_data.js?v={today}10" # Update record on 10AM daily


In [ ]:
def download_data(url):
    print(f"Fetching: {url}")
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Referer": "https://enquiry.indianrail.gov.in/ntes/"
    }
    
    res = requests.get(url, headers=headers, timeout=30)
    res.raise_for_status()
    return res.text

def save_to_json(stations):
    path = f"data/stations.json"
    with open(path, "w", encoding="utf-8") as f:
        json.dump(stations, f, indent=2, ensure_ascii=False)
    print(f"Saved {len(stations)} stations to {path}")


In [ ]:
def extract_stations(url):
    content = download_data(url)
    data = json.loads(content)
    stations = []

    for layer in data["layers"]:
        layer_depth = layer.get("depth")
        for feature in layer.get("features", []):
            attrs = feature.get("attributes", {})
            station = {
                    "Station": attrs.get("Station", ""),
                    "StationCode": attrs.get("StationCod", ""),
                    "ctrX": feature.get("ctrX"),
                    "ctrY": feature.get("ctrY"),
                    "layer_depth": layer_depth
                }
            stations.append(station)

        
    save_to_json(stations)
    return stations

In [ ]:
def parse_trains(content):
    # Extract the array content
    match = re.search(r'arrTrainList\s*=\s*\[(.*?)\]', content, re.DOTALL)
    if not match:
        raise ValueError("Could not find arrTrainList in response")
    
    # Extract all quoted strings
    raw_list = re.findall(r'"(.*?)"', match.group(1))
    
    trains = []
    for entry in raw_list:
        # Format is "00001- TRAIN NAME"
        parts = entry.split("- ", 1)
        if len(parts) == 2:
            trains.append({
                "TrainNo": parts[0].strip(),
                "TrainName": parts[1].strip()
            })
    
    return trains

In [ ]:
content = download_data()
trains = parse_trains(content)
# 
print(f"Total trains found: {len(trains)}")
print("Sample:", trains[:3])

# Save
with open("trains.json", "w", encoding="utf-8") as f:
    json.dump(trains, f, indent=2, ensure_ascii=False)

print("Saved to trains.json")

Fetching: https://enquiry.indianrail.gov.in/mntes/javascripts/train_data.js?v=2026021810
Total trains found: 10343
Sample: [{'TrainNo': '00001', 'TrainName': 'TEST FOR CCLMS'}, {'TrainNo': '00101', 'TrainName': 'PRYJ-CNB MAGHMELA SPL'}, {'TrainNo': '00102', 'TrainName': 'PRYJ CNB MAGHMELA SPL'}]
Saved to trains.json
